In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('./../')
sys.path.append('../../../reduction_phangs_hst/hstha_pipeline/modules/')

from astropy import units as au
from astropy.table import QTable
from astropy import stats

from tools_contsub_smoothregrid import *
from tools_contsub_misc import *

from astropy.convolution import convolve_fft, Gaussian2DKernel

from math import dist
from astropy import table

In [2]:
def get_std(data):
    data = data.flatten()   
    data = data[np.isfinite(data)]
    std_data = stats.mad_std(data, ignore_nan=True)
    std_data = stats.mad_std(data[(data>-std_data*10)&(data<std_data*10)], ignore_nan=True)
    return std_data

def get_noise_map(hdu):

    data = hdu.data
    header = hdu.header

    std_data = get_std(data)
    inital_value = std_data
    noise = np.random.normal(0, 1, data.shape)

    # pix_fwhm = 1.746
    pix_fwhm = 1.2
    pix_sigma = pix_fwhm/(2*np.sqrt(2*np.log(2)))
    kernal = Gaussian2DKernel(pix_sigma)

    noise_smoothed = convolve_fft(noise, kernal, preserve_nan=True, allow_huge=True)

    std = get_std(noise_smoothed)
    noise_smoothed = noise_smoothed / std

    hdu_noise = fits.PrimaryHDU(data=noise*inital_value, header=header)
    hdu_noise_smoothed = fits.PrimaryHDU(data=noise_smoothed*inital_value, header=header)

    std_noise = get_std(noise)
    std_noise_smoothed = get_std(noise_smoothed)

    return hdu_noise_smoothed

In [3]:
def add_noise(hdu_smoothed, hdu_noise_smoothed, target_noise=100, verbose=True):

    std = get_std(hdu_smoothed.data)
    std_noise = get_std(hdu_noise_smoothed.data)

    conv_noise = np.sqrt(target_noise**2 - std**2)
    # conv_noise = target_noise - std
    hdu_noise_smoothed.data = (conv_noise/std_noise) * hdu_noise_smoothed.data
    std_noise_scaled = get_std(hdu_noise_smoothed.data)

    hdu_smoothed_with_noise = hdu_smoothed.copy()
    hdu_smoothed_with_noise.data = hdu_smoothed_with_noise.data + hdu_noise_smoothed.data

    std_smoothed_with_noise = get_std(hdu_smoothed_with_noise.data)

    if verbose: 
        print(f'std: {std}, std_noise: {std_noise}, std_noise_scaled {std_noise_scaled}, std_smoothed_with_noise: {std_smoothed_with_noise}')

    return(hdu_smoothed_with_noise)

In [4]:
def convert_arcsec_parsecs(distance=10*au.Mpc, input_res=0.1*au.arcsec):
    input_res_pc = input_res.to('rad').value * distance.to('pc').value 
    return input_res_pc

In [5]:
rootdir = "/Users/abarnes/Dropbox/work/Smallprojects/galaxies/"
file_sample = f"{rootdir}/data_misc/sample_table/phangs_sample_table_v1p6.fits"
table_sample = QTable.read(file_sample)

# galaxies = ['ic5332', 'ngc1433', 'ngc3351', 'ngc4535', 
#             'ngc1087', 'ngc1512', 'ngc3627', 'ngc5068',
#             'ngc1300', 'ngc1566', 'ngc4254', 'ngc0628',
#             'ngc1365', 'ngc1672', 'ngc4303', 'ngc7496',
#             'ngc1385', 'ngc2835', 'ngc4321']

# galaxies_names = ['ic5332', 'ngc1433', 'ngc3351', 'ngc4535', 
#             'ngc1087', 'ngc1512', 'ngc3627', 'ngc5068',
#             'ngc1300', 'ngc1566', 'ngc4254', 'ngc628c',
#             'ngc1365n', 'ngc1672', 'ngc4303', 'ngc7496',
#             'ngc1385', 'ngc2835', 'ngc4321']

galaxies = ['ngc2835']
galaxies_names = ['ngc2835']

In [6]:
initial_res = 0.07*au.arcsec
distance_fix = 20.6265 *au.Mpc
resoution = convert_arcsec_parsecs(distance=distance_fix, input_res=initial_res)
print(f'At a distance of {distance_fix}, {initial_res} is {resoution:0.2f}pc...')

## ---- Main loop ---- ##
# Loop over all galaxies... 
# for gal, gal_name in zip([galaxies[0]], [galaxies_names[0]]):
for gal, gal_name in zip(galaxies, galaxies_names):

    # get distance
    distance = table_sample[table_sample['name'] == gal]['dist'][0]
    smooth_factor = distance_fix / distance # get smoothing factor to put all galaxies at the same distance
    target_res = initial_res * smooth_factor # get target resolution

    #  Define input file 
    inputfile = f"{rootdir}/data_hstha/{gal_name}/hst_contsub/{gal_name}_hst_ha.fits"
    print('-----------------------------------')
    print('Galaxy name:', gal_name)
    print(inputfile)

    # read in image
    hdu = fits.open(inputfile)[0]

    # get noise map - with some smoothing to match the correlation of the map
    hdu_noise = get_noise_map(hdu) 

    # smoothing image to 10pc resolution
    hdu_smoothed = get_smooth(hdu, initial_res, target_res)
    # smoothing noise to 10pc resolution 
    hdu_noise_smoothed = get_smooth(hdu_noise, initial_res, target_res)

    hdu_smoothed.header['BMAJ'] = target_res.to('deg').value
    hdu_smoothed.header['BMIN'] = target_res.to('deg').value
    hdu_smoothed.header['BPA'] = 0 

    # add noise to smoothed image    
    # 97 is the max noise level in the smoothed sample
    # hdu_smoothed_with_noise = add_noise(hdu_smoothed, hdu_noise, target_noise=100) # Apply original noise to smoothed image
    hdu_smoothed_with_noise = add_noise(hdu_smoothed, hdu_noise_smoothed, target_noise=100) # Apply smoothed noise to smoothed image

    # output file
    output_file_smoothed = inputfile.replace('.fits', '_10pc.fits')
    output_file_smoothed_with_noise = inputfile.replace('.fits', '_10pc_fixednoise.fits')
    print(f'Output file smoothed: {output_file_smoothed}')
    print(f'Output file smoothed with noise: {output_file_smoothed_with_noise}')

    # write image
    hdu_smoothed.writeto(output_file_smoothed, overwrite=True)
    hdu_smoothed_with_noise.writeto(output_file_smoothed_with_noise, overwrite=True)

At a distance of 20.6265 Mpc, 0.07 arcsec is 7.00pc...
-----------------------------------
Galaxy name: ngc2835
/Users/abarnes/Dropbox/work/Smallprojects/galaxies//data_hstha/ngc2835/hst_contsub/ngc2835_hst_ha.fits
[INFO] Pixel scale: 0.04 arcsec arcsec
[INFO] Initial Resolution: 0.07 arcsec arcsec
[INFO] Desired Resolution: 0.12 arcsec arcsec
[INFO] Convolution kernel: 0.10 arcsec arcsec
[INFO] Performing image convolution...
[INFO] Image convolution complete.
[INFO] Smoothing process completed.
[INFO] Pixel scale: 0.04 arcsec arcsec
[INFO] Initial Resolution: 0.07 arcsec arcsec
[INFO] Desired Resolution: 0.12 arcsec arcsec
[INFO] Convolution kernel: 0.10 arcsec arcsec
[INFO] Performing image convolution...
[INFO] Image convolution complete.
[INFO] Smoothing process completed.
std: 48.22249908231775, std_noise: 44.21551298617665, std_noise_scaled 87.604742636022, std_smoothed_with_noise: 102.80589467317854
Output file smoothed: /Users/abarnes/Dropbox/work/Smallprojects/galaxies//data_